In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove-text-file/glove.6B.100d.txt
/kaggle/input/files-of-data/valid_data.csv
/kaggle/input/files-of-data/train_data.csv
/kaggle/input/files-of-data/test_data.csv


In [2]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification
# import numpy as np

# # Load pre-trained GloVe embeddings
# glove_path = '/kaggle/input/glove-text-file/glove.6B.100d.txt'  # Path to your GloVe embeddings file
# embedding_matrix[index] = np.reshape(glove_embeddings[word], (1, -1))

# glove_embeddings = {}

# with open(glove_path, 'r', encoding='utf-8') as f:
#     for line in f:
#         values = line.strip().split(' ')
#         word = values[0]
#         embeddings = np.asarray(values[1:], dtype=np.float32)
#         glove_embeddings[word] = embeddings

# # Load pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# # Initialize BERT model's embedding layer with GloVe embeddings
# vocab_size = tokenizer.vocab_size
# embedding_dim = model.config.hidden_size

# embedding_matrix = np.zeros((vocab_size, embedding_dim))

# for word, index in tokenizer.get_vocab().items():
#     if word in glove_embeddings:
#         embedding_matrix[index] = glove_embeddings[word]

# model.bert.embeddings.word_embeddings.weight.data = torch.tensor(embedding_matrix)

# # Example user query
# user_query = "machine learning"

# # Tokenize user query
# query_tokens = tokenizer.tokenize(user_query)
# query_ids = tokenizer.convert_tokens_to_ids(query_tokens)
# query_ids = tokenizer.build_inputs_with_special_tokens(query_ids)

# # Convert query to tensor
# query_tensor = torch.tensor([query_ids])

# # Forward pass through the BERT model
# outputs = model(query_tensor)

# # Get the predicted probabilities and predicted label
# probs = torch.softmax(outputs[0], dim=1)
# predicted_label = torch.argmax(probs, dim=1).item()

# # Convert label index to label text
# label_map = {0: "Not Relevant", 1: "Relevant"}  # Modify as per your task
# predicted_label_text = label_map[predicted_label]

# # Print results
# print("User Query:", user_query)
# print("Predicted Label:", predicted_label_text)
# print("Predicted Probabilities:", probs.tolist())


In [3]:
!pip install --upgrade keras tensorflow

In [4]:
!pip install keras

In [5]:
import keras
print(keras.__version__) 


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

2.12.0


In [6]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import spacy
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import (LSTM, 
                                      Embedding, 
                                      BatchNormalization,
                                      Dense, 
                                      TimeDistributed, 
                                      Dropout, 
                                      Bidirectional,
                                      Flatten, 
                                      GlobalMaxPool1D)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import (precision_score, 
                             recall_score, 
                             f1_score, 
                             classification_report,
                             accuracy_score)


In [7]:
test_df = pd.read_csv("/kaggle/input/files-of-data/test_data.csv",nrows=500 )
test_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,soimort/you-get,src/you_get/extractors/youtube.py,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...",python,"def get_vid_from_url(url):\n """"""Extract...","['def', 'get_vid_from_url', '(', 'url', ')', '...",Extracts video ID from URL.,"['Extracts', 'video', 'ID', 'from', 'URL', '.']",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test
1,soimort/you-get,src/you_get/extractors/miomio.py,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...",python,"def sina_xml_to_url_list(xml_data):\n """"""st...","['def', 'sina_xml_to_url_list', '(', 'xml_data...",str->list\n Convert XML to URL List.\n F...,"['str', '-', '>', 'list', 'Convert', 'XML', 't...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test
2,soimort/you-get,src/you_get/extractors/fc2video.py,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....",python,"def makeMimi(upid):\n """"""From http://cdn37....","['def', 'makeMimi', '(', 'upid', ')', ':', 'st...",From http://cdn37.atwikiimg.com/sitescript/pub...,"['From', 'http', ':', '//', 'cdn37', '.', 'atw...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test
3,soimort/you-get,src/you_get/extractors/fc2video.py,fc2video_download,"def fc2video_download(url, output_dir = '.', m...",python,"def fc2video_download(url, output_dir = '.', m...","['def', 'fc2video_download', '(', 'url', ',', ...",wrapper,['wrapper'],b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test
4,soimort/you-get,src/you_get/extractors/dailymotion.py,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...",python,"def dailymotion_download(url, output_dir='.', ...","['def', 'dailymotion_download', '(', 'url', ',...",Downloads Dailymotion videos by URL.,"['Downloads', 'Dailymotion', 'videos', 'by', '...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test


In [8]:
train_df = pd.read_csv("/kaggle/input/files-of-data/train_data.csv", nrows=1000)
train_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,ageitgey/face_recognition,examples/face_recognition_knn.py,train,"def train(train_dir, model_save_path=None, n_n...",python,"def train(train_dir, model_save_path=None, n_n...","['def', 'train', '(', 'train_dir', ',', 'model...",Trains a k-nearest neighbors classifier for fa...,"['Trains', 'a', 'k', '-', 'nearest', 'neighbor...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train
1,ageitgey/face_recognition,examples/face_recognition_knn.py,predict,"def predict(X_img_path, knn_clf=None, model_pa...",python,"def predict(X_img_path, knn_clf=None, model_pa...","['def', 'predict', '(', 'X_img_path', ',', 'kn...",Recognizes faces in given image using a traine...,"['Recognizes', 'faces', 'in', 'given', 'image'...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train
2,ageitgey/face_recognition,examples/face_recognition_knn.py,show_prediction_labels_on_image,"def show_prediction_labels_on_image(img_path, ...",python,"def show_prediction_labels_on_image(img_path, ...","['def', 'show_prediction_labels_on_image', '('...",Shows the face recognition results visually.\n...,"['Shows', 'the', 'face', 'recognition', 'resul...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train
3,ageitgey/face_recognition,face_recognition/api.py,_rect_to_css,"def _rect_to_css(rect):\n """"""\n Convert ...",python,"def _rect_to_css(rect):\n """"""\n Convert ...","['def', '_rect_to_css', '(', 'rect', ')', ':',...",Convert a dlib 'rect' object to a plain tuple ...,"['Convert', 'a', 'dlib', 'rect', 'object', 'to...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train
4,ageitgey/face_recognition,face_recognition/api.py,_trim_css_to_bounds,"def _trim_css_to_bounds(css, image_shape):\n ...",python,"def _trim_css_to_bounds(css, image_shape):\n ...","['def', '_trim_css_to_bounds', '(', 'css', ','...","Make sure a tuple in (top, right, bottom, left...","['Make', 'sure', 'a', 'tuple', 'in', '(', 'top...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train


In [9]:
balance_counts = train_df.groupby('docstring')['docstring'].agg('count').values
balance_counts

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [10]:
balance_counts = test_df.groupby('url')['url'].agg('count').values
balance_counts

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [11]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['docstring'],
    y=[balance_counts[0]],
    name='docstring',
    text=[balance_counts[0]],
    textposition='auto',
))
fig.add_trace(go.Bar(
    x=['url'],
    y=[balance_counts[1]],
    name='url',
    text=[balance_counts[1]],
    textposition='auto',
))
fig.update_layout(
    title='<span style="font-size:32px; font-family:Times New Roman">Dataset distribution by target</span>'
)
fig.show()

In [12]:
# Special thanks to https://www.kaggle.com/tanulsingh077 for this function
def clean_docstring(docstring):
    '''Make docstring lowercase, remove docstring in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    docstring = str(docstring).lower()
    docstring = re.sub('\[.*?\]', '', docstring)
    docstring = re.sub('https?://\S+|www\.\S+', '', docstring)
    docstring = re.sub('<.*?>+', '', docstring)
    docstring = re.sub('[%s]' % re.escape(string.punctuation), '', docstring)
    docstring = re.sub('\n', '', docstring)
    docstring = re.sub('\w*\d\w*', '', docstring)
    return docstring  

In [13]:
train_df['docstring_clean'] = train_df['docstring'].apply(clean_docstring)
train_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition,docstring_clean
0,ageitgey/face_recognition,examples/face_recognition_knn.py,train,"def train(train_dir, model_save_path=None, n_n...",python,"def train(train_dir, model_save_path=None, n_n...","['def', 'train', '(', 'train_dir', ',', 'model...",Trains a k-nearest neighbors classifier for fa...,"['Trains', 'a', 'k', '-', 'nearest', 'neighbor...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,trains a knearest neighbors classifier for fac...
1,ageitgey/face_recognition,examples/face_recognition_knn.py,predict,"def predict(X_img_path, knn_clf=None, model_pa...",python,"def predict(X_img_path, knn_clf=None, model_pa...","['def', 'predict', '(', 'X_img_path', ',', 'kn...",Recognizes faces in given image using a traine...,"['Recognizes', 'faces', 'in', 'given', 'image'...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,recognizes faces in given image using a traine...
2,ageitgey/face_recognition,examples/face_recognition_knn.py,show_prediction_labels_on_image,"def show_prediction_labels_on_image(img_path, ...",python,"def show_prediction_labels_on_image(img_path, ...","['def', 'show_prediction_labels_on_image', '('...",Shows the face recognition results visually.\n...,"['Shows', 'the', 'face', 'recognition', 'resul...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,shows the face recognition results visually ...
3,ageitgey/face_recognition,face_recognition/api.py,_rect_to_css,"def _rect_to_css(rect):\n """"""\n Convert ...",python,"def _rect_to_css(rect):\n """"""\n Convert ...","['def', '_rect_to_css', '(', 'rect', ')', ':',...",Convert a dlib 'rect' object to a plain tuple ...,"['Convert', 'a', 'dlib', 'rect', 'object', 'to...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,convert a dlib rect object to a plain tuple in...
4,ageitgey/face_recognition,face_recognition/api.py,_trim_css_to_bounds,"def _trim_css_to_bounds(css, image_shape):\n ...",python,"def _trim_css_to_bounds(css, image_shape):\n ...","['def', '_trim_css_to_bounds', '(', 'css', ','...","Make sure a tuple in (top, right, bottom, left...","['Make', 'sure', 'a', 'tuple', 'in', '(', 'top...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,make sure a tuple in top right bottom left ord...


In [14]:
test_df['docstring_clean'] = test_df['docstring'].apply(clean_docstring)
test_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition,docstring_clean
0,soimort/you-get,src/you_get/extractors/youtube.py,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...",python,"def get_vid_from_url(url):\n """"""Extract...","['def', 'get_vid_from_url', '(', 'url', ')', '...",Extracts video ID from URL.,"['Extracts', 'video', 'ID', 'from', 'URL', '.']",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,extracts video id from url
1,soimort/you-get,src/you_get/extractors/miomio.py,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...",python,"def sina_xml_to_url_list(xml_data):\n """"""st...","['def', 'sina_xml_to_url_list', '(', 'xml_data...",str->list\n Convert XML to URL List.\n F...,"['str', '-', '>', 'list', 'Convert', 'XML', 't...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,strlist convert xml to url list from bil...
2,soimort/you-get,src/you_get/extractors/fc2video.py,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....",python,"def makeMimi(upid):\n """"""From http://cdn37....","['def', 'makeMimi', '(', 'upid', ')', ':', 'st...",From http://cdn37.atwikiimg.com/sitescript/pub...,"['From', 'http', ':', '//', 'cdn37', '.', 'atw...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,from also
3,soimort/you-get,src/you_get/extractors/fc2video.py,fc2video_download,"def fc2video_download(url, output_dir = '.', m...",python,"def fc2video_download(url, output_dir = '.', m...","['def', 'fc2video_download', '(', 'url', ',', ...",wrapper,['wrapper'],b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,wrapper
4,soimort/you-get,src/you_get/extractors/dailymotion.py,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...",python,"def dailymotion_download(url, output_dir='.', ...","['def', 'dailymotion_download', '(', 'url', ',...",Downloads Dailymotion videos by URL.,"['Downloads', 'Dailymotion', 'videos', 'by', '...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,downloads dailymotion videos by url


In [15]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(docstring):
    docstring = ' '.join(word for word in docstring.split(' ') if word not in stop_words)
    return docstring
    
test_df['docstring_clean'] = test_df['docstring_clean'].apply(remove_stopwords)
train_df['docstring_clean'] = train_df['docstring_clean'].apply(remove_stopwords)

In [16]:
stemmer = nltk.SnowballStemmer("english")

def stemm_docstring(docstring):
    docstring = ' '.join(stemmer.stem(word) for word in docstring.split(' '))
    return docstring

In [17]:
test_df['docstring_clean'] = test_df['docstring_clean'].apply(stemm_docstring)


In [18]:
train_df['docstring_clean'] = train_df['docstring_clean'].apply(stemm_docstring)

In [19]:
def preprocess_data(docstring):
    # Clean puntuation, urls, and so on
    docstring = clean_docstring(docstring)
    # Remove stopwords
    docstring = ' '.join(word for word in docstring.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    docstring = ' '.join(stemmer.stem(word) for word in docstring.split(' '))
    
    return docstring

In [20]:
test_df['docstring_clean'] = test_df['docstring_clean'].apply(preprocess_data)


In [21]:
train_df['docstring_clean'] = train_df['docstring_clean'].apply(preprocess_data)

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_df['url'])


train_df['url_encoded'] = le.transform(train_df['url'])
train_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition,docstring_clean,url_encoded
0,ageitgey/face_recognition,examples/face_recognition_knn.py,train,"def train(train_dir, model_save_path=None, n_n...",python,"def train(train_dir, model_save_path=None, n_n...","['def', 'train', '(', 'train_dir', ',', 'model...",Trains a k-nearest neighbors classifier for fa...,"['Trains', 'a', 'k', '-', 'nearest', 'neighbor...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,train knearest neighbor classifi face recognit...,2
1,ageitgey/face_recognition,examples/face_recognition_knn.py,predict,"def predict(X_img_path, knn_clf=None, model_pa...",python,"def predict(X_img_path, knn_clf=None, model_pa...","['def', 'predict', '(', 'X_img_path', ',', 'kn...",Recognizes faces in given image using a traine...,"['Recognizes', 'faces', 'in', 'given', 'image'...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,recogn face given imag use train knn classifi ...,0
2,ageitgey/face_recognition,examples/face_recognition_knn.py,show_prediction_labels_on_image,"def show_prediction_labels_on_image(img_path, ...",python,"def show_prediction_labels_on_image(img_path, ...","['def', 'show_prediction_labels_on_image', '('...",Shows the face recognition results visually.\n...,"['Shows', 'the', 'face', 'recognition', 'resul...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,show face recognit result visual param imgp...,1
3,ageitgey/face_recognition,face_recognition/api.py,_rect_to_css,"def _rect_to_css(rect):\n """"""\n Convert ...",python,"def _rect_to_css(rect):\n """"""\n Convert ...","['def', '_rect_to_css', '(', 'rect', ')', ':',...",Convert a dlib 'rect' object to a plain tuple ...,"['Convert', 'a', 'dlib', 'rect', 'object', 'to...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,convert dlib rect object plain tupl top right ...,7
4,ageitgey/face_recognition,face_recognition/api.py,_trim_css_to_bounds,"def _trim_css_to_bounds(css, image_shape):\n ...",python,"def _trim_css_to_bounds(css, image_shape):\n ...","['def', '_trim_css_to_bounds', '(', 'css', ','...","Make sure a tuple in (top, right, bottom, left...","['Make', 'sure', 'a', 'tuple', 'in', '(', 'top...",c96b010c02f15e8eeb0f71308c641179ac1f19bb,https://github.com/ageitgey/face_recognition/b...,train,make sure tupl top right bottom left order wit...,8


In [23]:
le = LabelEncoder()
le.fit(test_df['url'])


test_df['url_encoded'] = le.transform(test_df['url'])
test_df.head()

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition,docstring_clean,url_encoded
0,soimort/you-get,src/you_get/extractors/youtube.py,YouTube.get_vid_from_url,"def get_vid_from_url(url):\n """"""Extract...",python,"def get_vid_from_url(url):\n """"""Extract...","['def', 'get_vid_from_url', '(', 'url', ')', '...",Extracts video ID from URL.,"['Extracts', 'video', 'ID', 'from', 'URL', '.']",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,extract video id url,490
1,soimort/you-get,src/you_get/extractors/miomio.py,sina_xml_to_url_list,"def sina_xml_to_url_list(xml_data):\n """"""st...",python,"def sina_xml_to_url_list(xml_data):\n """"""st...","['def', 'sina_xml_to_url_list', '(', 'xml_data...",str->list\n Convert XML to URL List.\n F...,"['str', '-', '>', 'list', 'Convert', 'XML', 't...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,strlist convert xml url list biligrab,471
2,soimort/you-get,src/you_get/extractors/fc2video.py,makeMimi,"def makeMimi(upid):\n """"""From http://cdn37....",python,"def makeMimi(upid):\n """"""From http://cdn37....","['def', 'makeMimi', '(', 'upid', ')', ':', 'st...",From http://cdn37.atwikiimg.com/sitescript/pub...,"['From', 'http', ':', '//', 'cdn37', '.', 'atw...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,also,464
3,soimort/you-get,src/you_get/extractors/fc2video.py,fc2video_download,"def fc2video_download(url, output_dir = '.', m...",python,"def fc2video_download(url, output_dir = '.', m...","['def', 'fc2video_download', '(', 'url', ',', ...",wrapper,['wrapper'],b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,wrapper,465
4,soimort/you-get,src/you_get/extractors/dailymotion.py,dailymotion_download,"def dailymotion_download(url, output_dir='.', ...",python,"def dailymotion_download(url, output_dir='.', ...","['def', 'dailymotion_download', '(', 'url', ',...",Downloads Dailymotion videos by URL.,"['Downloads', 'Dailymotion', 'videos', 'by', '...",b746ac01c9f39de94cac2d56f665285b0523b974,https://github.com/soimort/you-get/blob/b746ac...,test,download dailymot video url,463


In [24]:
test_x = test_df['docstring_clean']
test_y = test_df['url_encoded']

print(len(test_x), len(test_y))

500 500


In [25]:
train_x = train_df['docstring_clean']
train_y = train_df['url_encoded']

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(train_x)


CountVectorizer()

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(test_x)

CountVectorizer()

In [28]:
x_test_dtm = vect.transform(test_x)
x_train_dtm = vect.transform(train_x)

In [29]:
vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(x_train_dtm)
x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

x_train_tfidf

<1000x1570 sparse matrix of type '<class 'numpy.float64'>'
	with 13019 stored elements in Compressed Sparse Row format>

In [31]:
texts = train_df['docstring_clean']
target = train_df['url_encoded']

In [32]:
# Calculate the length of our vocabulary
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

3808

In [33]:
def embed(corpus): 
    return word_tokenizer.texts_to_sequences(corpus)

longest_train = max(texts, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))

train_padded_sentences = pad_sequences(
    embed(texts), 
    length_long_sentence, 
    padding='post'
)

train_padded_sentences

array([[ 105, 2050,  931, ...,    0,    0,    0],
       [ 821,  150,   28, ...,    0,    0,    0],
       [ 482,  150, 1399, ...,    0,    0,    0],
       ...,
       [  33,   23,  171, ...,    0,    0,    0],
       [ 338,   28,    3, ...,    0,    0,    0],
       [ 215, 3807,    2, ...,    0,    0,    0]], dtype=int32)

In [34]:
embeddings_dictionary = dict()
embedding_dim = 100

# Load GloVe 100D embeddings
with open('/kaggle/input/glove-text-file/glove.6B.100d.txt') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions

# embeddings_dictionary

In [35]:
 #Now we will load embedding vectors of those words that appear in the
# Glove dictionary. Others will be initialized to 0.

embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.096096  ,  0.30311999, -0.14951   , ...,  0.2158    ,
        -1.03349996, -0.35038999],
       [ 0.10483   , -0.20148   ,  0.20761   , ...,  0.56396002,
         0.070056  , -0.21981999],
       ...,
       [-0.74070001,  0.26899001,  0.79694998, ...,  0.14331   ,
        -0.066576  , -0.86271   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [36]:
# Create a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(x_train_dtm, train_y)

MultinomialNB()

In [37]:
# Make class anf probability predictions
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

In [38]:
# # calculate accuracy of class predictions
# from sklearn import metrics
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# import numpy as np

# confusion = confusion_matrix(test_y, y_pred_class)
# print(confusion)

# print(metrics.accuracy_score(test_y, y_pred_class))

# conf_matrix(metrics.confusion_matrix(test_y, y_pred_class))

In [39]:
# # Calculate AUC
# metrics.roc_auc_score(test_y, y_pred_prob, multi_class='ovr')

In [40]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])

In [41]:
# Fit the pipeline with the data
pipe.fit(train_x, train_y)

y_pred_class = pipe.predict(test_x)

# print(metrics.accuracy_score(test_y, y_pred_class))

# conf_matrix(metrics.confusion_matrix(test_y, y_pred_class))

In [42]:
import xgboost as xgb

pipe = Pipeline([
    ('bow', CountVectorizer()), 
    ('tfid', TfidfTransformer()),  
    ('model', xgb.XGBClassifier(
        learning_rate=0.1,
        max_depth=7,
        n_estimators=80,
        use_label_encoder=False,
        eval_metric='auc',
        # colsample_bytree=0.8,
        # subsample=0.7,
        # min_child_weight=5,
    ))
])

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



In [43]:
# Fit the pipeline with the data
pipe.fit(train_x, train_y)

y_pred_class = pipe.predict(test_x)
y_pred_train = pipe.predict(train_x)

print('Train: {}'.format(metrics.accuracy_score(train_y, y_pred_train)))
print('Test: {}'.format(metrics.accuracy_score(test_y, y_pred_class)))

# conf_matrix(metrics.confusion_matrix(test_y, y_pred_class))

NameError: name 'metrics' is not defined

**BERT**

In [47]:
# import torch
# from transformers import BertTokenizer, BertModel
# import numpy as np

# # Load pre-trained GloVe embeddings
# glove_path = '/kaggle/input/glove-text-file/glove.6B.100d.txt'  # Path to your GloVe embeddings file
# glove_embeddings = {}

# with open(glove_path, 'r', encoding='utf-8') as f:
#     for line in f:
#         values = line.strip().split(' ')
#         word = values[0]
# #         embeddings = np.asarray(values[1:], dtype=np.float32)
#         glove_embeddings[word] = embeddings

# # Load pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

# # Initialize projection layer
# projection = torch.nn.Linear(100, 768)

# # Example input
# input_text = "This is an example sentence."

# # Tokenize input text
# input_ids = tokenizer.encode(input_text, add_special_tokens=True)
# input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

# # Convert GloVe embeddings to tensor
# embeddings_tensor = torch.tensor([glove_embeddings[word] for word in tokenizer.get_vocab() if word in glove_embeddings])

# # Project GloVe embeddings to match BERT hidden size
# projected_embeddings = projection(embeddings_tensor)

# # Initialize BERT model's embedding layer with projected GloVe embeddings
# model.embeddings.word_embeddings.weight.data = projected_embeddings

# # Forward pass through the model
# outputs = model(input_ids)

# # Get the contextualized word embeddings for each token
# contextual_embeddings = outputs.last_hidden_state

# # Print the contextualized word embeddings
# print("Input Text:", input_text)
# print("Contextualized Word Embeddings:", contextual_embeddings)


NameError: name 'embeddings' is not defined

In [44]:
!pip install transformers

In [45]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.8 MB/s eta 0:00:0000:01m0:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [46]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

In [47]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
except:
    strategy = tf.distribute.get_strategy()
    
print('Number of replicas in sync: ', strategy.num_replicas_in_sync)

Number of replicas in sync:  1


In [48]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def bert_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

In [50]:
texts = test_df['docstring_clean']
target = test_df['url_encoded']

train_input_ids, train_attention_masks = bert_encode(texts,60)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



In [51]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

def create_model(bert_model):
    
    input_ids = tf.keras.Input(shape=(60,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(60,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [52]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-large-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [53]:
model = create_model(bert_model)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  335141888   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 60,                                            

In [54]:
model = create_model(bert_model)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  335141888   ['input_3[0][0]',                
                                thPoolingAndCrossAt               'input_4[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 60,                                          

In [ ]:
model

In [62]:
import matplotlib.pyplot as plt

# Extract loss and accuracy values from the training history
loss = history['loss']
val_loss = history['val_loss']
accuracy = history['accuracy']
val_accuracy = history['val_accuracy']

# Plot the learning curves
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Learning Curves')
plt.legend()
plt.show()


NameError: name 'history' is not defined

In [55]:
plot_learning_curves(history, [['loss', 'val_loss'],['accuracy', 'val_accuracy']])

NameError: name 'plot_learning_curves' is not defined